In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader





In [2]:
#data pre processing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081))
])


In [6]:
#load data
train_dataset=datasets.MNIST(root='./data',train = True, download=True, transform=transform)
test_dataset=datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader=DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader=DataLoader(train_dataset, batch_size=64, shuffle=True)

